1. Load batch data
2. Loop cases
3. Push results back to staging SQLite

In [ ]:
import sqlite3
import logging
import pandas as pd

from text2graph.utils import get_output_info, get_eta

logging.basicConfig(level=logging.INFO)

In [ ]:
run_name = "olvi"
df = pd.read_parquet("data/formation_sample.parquet.gzip")

In [ ]:
get_eta(
    eval_db="data/eval.db",
    test_set="data/formation_sample.parquet.gzip",
    run_name=run_name,
    n_workers=7,
)

In [ ]:
conn = sqlite3.connect("data/eval.db")
conn.close()

Calculate the estimated finish time for the run

Parsing output to triplet format

In [ ]:
def parse_output(output: str) -> list[dict]:
    """Parse the json output from the record."""
    if not output:
        return []
    return get_output_info(output, ["message", "content"])

Drop obsolete testing tables

In [ ]:
# Drop test table
drop_table_name = "test_olvi"
cursor.execute(f"DROP TABLE IF EXISTS {drop_table_name}")

### Extract output data

In [ ]:
db_df = pd.read_sql_query("SELECT * FROM olvi;", conn)

In [ ]:
db_df["triplets"] = db_df["output"].apply(parse_output)

In [ ]:
db_df_sel = db_df[~db_df.triplets.map(bool)]